# CLINICAL TRIAL ANALYTICAL DASHBOARD

In case you can not access to ddbb due to run it through a jupyter notebook. You can proceed with the next steps to grant acces to ddbb:
1. Get inside the mysql docker to modify privileges (use the root's passord, check .env file send by email)
```bash
docker exec -it mysql-clinical-db  mysql -u root -p
```
2.  Inside MySQL (modify the tu_password to the root's passord, check .env file send by email):
```sql
GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' IDENTIFIED BY 'tu_password';
GRANT ALL PRIVILEGES ON *.* TO 'root'@'172.%' IDENTIFIED BY 'tu_password';  -- Cubre Docker IPs
FLUSH PRIVILEGES;
SELECT user, host FROM mysql.user WHERE user='root';  -- Verifica '%'
EXIT;
```
It should print on screen:
```
+------+-----------+
| user | host      |
+------+-----------+
| root | %         |
| root | localhost |
+------+-----------+
```


3.  Restart dockers:
```bash
docker compose down && docker compose up -d
```

4.  Connect to ddbb using root as user:

        os.environ['DB_HOST'] = '127.0.0.1' # e.g., 'your_remote_db_host.com' or '127.0.0.1' if running locally

        os.environ['DB_PORT'] = '3306' # Your specific MySQL port

        os.environ['DB_USER'] = 'root' # Your MySQL username

        os.environ['DB_PASSWORD'] = 'rootpass' # Your MySQL password

        os.environ['DB_NAME'] = 'clinicaltrials' # Your database name





## 1. IMPORTS


In [26]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
from pathlib import Path
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from loguru import logger
# Siempre usa display() en lugar de print()
from IPython.display import display


## 2. FUNCTIONS


In [14]:

# Database connection with retry for the seed check
def get_db_connection_with_retry(retries=3, delay=1):
    for attempt in range(retries):
        try:
            connection = mysql.connector.connect(
                host=os.getenv('DB_HOST', 'mysql'),
                port=int(os.getenv('DB_PORT', 3306)),
                user=os.getenv('DB_USER', 'user'),
                password=os.getenv('DB_PASSWORD', 'pass'),
                database=os.getenv('DB_NAME', 'clinicaltrials')
            )
            return connection
        except Error as e:
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logger.error(f"Database connection failed after {retries} attempts: {e}")
                return None
    return None


In [15]:
tables_ddbb= ['studies', 'conditions', 'interventions', 'outcomes', 'sponsors', 'locations', 'study_design']


In [16]:
for table in tables_ddbb:
    print(table)

studies
conditions
interventions
outcomes
sponsors
locations
study_design


## 3. DATA QUERIES (SQL)

Next text is generated by AI when asking to group all sql queries performed in the clinical dashboard platform

SQL Queries Extracted

Query: SELECT COUNT(*) as total FROM studies

Counts total studies; used to detect if DB is empty (0 rows → empty).​
Row Counts

Query: SELECT COUNT(*) as count FROM {selected_table} (dynamic: studies, conditions, etc.)

Fetches row count for any selected table; displays as metric.​
Describe Tables

Query: DESCRIBE {selected_table}

Lists columns/types for data availability analysis.​
Data Availability

Query: SELECT COUNT(*) as total, SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) as null_count FROM {selected_table}

Per-column null/total counts → availability % (e.g., non-null/total*100).​
Complete Records

Query: SELECT COUNT(*) as complete_count FROM {selected_table} WHERE {col1} IS NOT NULL AND {col2} IS NOT NULL ...

Counts rows with NO nulls across all columns.​
Summary Stats

Query: SELECT COUNT(*) as total, COUNT(DISTINCT {col}) as distinct_count FROM {selected_table} WHERE {col} IS NOT NULL

Total vs unique non-null values per key column (status, phase, etc.).​
Top Values

Query: SELECT {col}, COUNT(*) as count FROM {selected_table} WHERE {col}IS NOT NULL GROUP BY{col} ORDER BY count DESC LIMIT 5

Top 5 frequent values per column (e.g., status, condition_name).​
Studies Distributions

Queries:

    SELECT status, COUNT(*) as count FROM studies WHERE status IS NOT NULL GROUP BY status ORDER BY count DESC

    SELECT phase, COUNT(*) as count FROM studies WHERE phase IS NOT NULL GROUP BY phase ORDER BY count DESC

    SELECT gender, COUNT(*) as count FROM studies WHERE gender IS NOT NULL GROUP BY gender ORDER BY count DESC

Aggregates for bar/pie charts on status, phase, gender.​
Enrollment Stats

Queries:

    SELECT enrollment FROM studies WHERE enrollment IS NOT NULL (for histogram)

    SELECT AVG(enrollment) as avg_enrollment, MIN(enrollment) as min, MAX(enrollment) as max, COUNT(*) as total_studies FROM studies WHERE enrollment IS NOT NULL

    SELECT COUNT(*) as zero_enrollment FROM studies WHERE enrollment = 0

Stats (avg/min/max/zeros) for enrollment metrics/histogram.​
Top Entities

Queries:

    SELECT condition_name, COUNT(*) as count FROM conditions GROUP BY condition_name ORDER BY count DESC LIMIT 15

    SELECT intervention_type, COUNT(*) as count FROM interventions WHERE intervention_type IS NOT NULL GROUP BY intervention_type ORDER BY count DESC

    SELECT outcome_type, COUNT(*) as count FROM outcomes WHERE outcome_type IS NOT NULL GROUP BY outcome_type ORDER BY count DESC

    SELECT agency, COUNT(*) as count FROM sponsors GROUP BY agency ORDER BY count DESC LIMIT 15

    SELECT country, COUNT(*) as count FROM locations WHERE country IS NOT NULL GROUP BY country ORDER BY count DESC LIMIT 15

    SELECT allocation, COUNT(*) as count FROM study_design WHERE allocation IS NOT NULL GROUP BY allocation ORDER BY count DESC

    SELECT primary_purpose, COUNT(*) as count FROM study_design WHERE primary_purpose IS NOT NULL GROUP BY primary_purpose ORDER BY count DESC

Top-N (5-15) by frequency for viz (bars, lollipops).​
Time Trends

Queries:

    SELECT YEAR(start_date) as year, COUNT(*) as study_count FROM studies WHERE start_date IS NOT NULL GROUP BY YEAR(start_date) ORDER BY year

    SELECT YEAR(completion_date) as year, COUNT(*) as completed_count FROM studies WHERE completion_date IS NOT NULL GROUP BY YEAR(completion_date) ORDER BY year

    SELECT YEAR(start_date) as year, AVG(enrollment) as avg_enrollment, COUNT(*) as study_count FROM studies WHERE start_date IS NOT NULL AND enrollment IS NOT NULL GROUP BY YEAR(start_date) ORDER BY year

    SELECT YEAR(start_date) as year, phase, COUNT(*) as count FROM studies WHERE start_date IS NOT NULL AND phase IS NOT NULL GROUP BY YEAR(start_date), phase ORDER BY year, phase

    SELECT YEAR(s.start_date) as year, c.condition_name, COUNT(*) as count FROM studies s JOIN conditions c ON s.study_id = c.study_id WHERE s.start_date IS NOT NULL GROUP BY YEAR(s.start_date), c.condition_name HAVING COUNT(*) >= 3 ORDER BY year, count DESC

Yearly trends (line/area/bar) for starts, completions, enrollment avg, phase evolution, top conditions.​
Search Studies

Query: SELECT * FROM studies WHERE title LIKE %s OR description LIKE %s LIMIT 50

Full-text search on title/description; params prevent injection.​


## 1. IMPORTS


## 3. DATAFRAMES GENERATION


In [ ]:
connection = get_db_connection_with_retry()
dict_df_clinical_trials = {}
if connection:
    try:

        for table in tables_ddbb:
            query = f'SELECT * FROM {table}'
            dict_df_clinical_trials[table] = pd.read_sql_query(query, connection)
            logger.success(f"Data from '{table}' table loaded successfully. Number of records: {len(dict_df_clinical_trials[table])}")
        logger.success("All Data from ddbb loaded successfully.")
    except Error as e:
        logger.error(f"Error reading data from MySQL: {e}")
    finally:
        if connection.is_connected():
            connection.close()
            logger.info("MySQL connection closed.")
else:
    logger.error("Failed to connect to MySQL database.")

/tmp/ipykernel_276868/550113246.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dict_df_clinical_trials[table] = pd.read_sql_query(query, connection)
2026-02-03 09:40:55.150 | SUCCESS  | __main__:<module>:9 - Data from 'studies' table loaded successfully. Number of records: 10000
2026-02-03 09:40:55.188 | SUCCESS  | __main__:<module>:9 - Data from 'conditions' table loaded successfully. Number of records: 17750
2026-02-03 09:40:55.234 | SUCCESS  | __main__:<module>:9 - Data from 'interventions' table loaded successfully. Number of records: 16715
2026-02-03 09:40:55.416 | SUCCESS  | __main__:<module>:9 - Data from 'outcomes' table loaded successfully. Number of records: 61766
2026-02-03 09:40:55.456 | SUCCESS  | __main__:<module>:9 - Data from 'sponsors' table loaded successfully. Number of records: 15861
2026-02-03 09:40:55.615

## 4. DATA EXPLORATORY ANALYSIS

### 4.1. Statistical Analysis

In [27]:
for table in tables_ddbb:
    print (f"# # # # # # # {table}   # # # # # # #")
    display(dict_df_clinical_trials[table].describe())
            

# # # # # # # studies   # # # # # # #


,study_id,enrollment,created_at,updated_at
count,10000.00000,9.872000e+03,10000,10000
mean,5000.50000,2.066886e+03,2026-02-02 19:29:38.556600,2026-02-02 19:29:38.556600
min,1.00000,0.000000e+00,2026-02-02 19:29:25,2026-02-02 19:29:25
25%,2500.75000,3.000000e+01,2026-02-02 19:29:32,2026-02-02 19:29:32
50%,5000.50000,6.800000e+01,2026-02-02 19:29:38,2026-02-02 19:29:38
75%,7500.25000,1.900000e+02,2026-02-02 19:29:46,2026-02-02 19:29:46
max,10000.00000,4.238504e+06,2026-02-02 19:29:52,2026-02-02 19:29:52
std,2886.89568,5.925177e+04,NaN,NaN


# # # # # # # conditions   # # # # # # #


,condition_id,study_id
count,17750.000000,17750.000000
mean,8875.500000,4962.439211
std,5124.127975,2875.612055
min,1.000000,1.000000
25%,4438.250000,2485.000000
50%,8875.500000,4972.000000
75%,13312.750000,7408.000000
max,17750.000000,10000.000000


# # # # # # # interventions   # # # # # # #


...

,sponsor_id,study_id
count,15861.000000,15861.000000
mean,7931.000000,4972.140975
std,4578.820645,2904.195303
min,1.000000,1.000000
25%,3966.000000,2461.000000
50%,7931.000000,5000.000000
75%,11896.000000,7460.000000
max,15861.000000,10000.000000


# # # # # # # locations   # # # # # # #


,location_id,study_id
count,56916.000000,56916.000000
mean,28458.500000,5297.656406
std,16430.378298,2936.649709
min,1.000000,1.000000
25%,14229.750000,2851.000000
50%,28458.500000,5353.000000
75%,42687.250000,7848.000000
max,56916.000000,10000.000000


# # # # # # # study_design   # # # # # # #


,design_id,study_id
count,9851.000000,9851.000000
mean,4926.000000,5008.067303
std,2843.883085,2888.100171
min,1.000000,1.000000
25%,2463.500000,2507.500000
50%,4926.000000,5016.000000
75%,7388.500000,7509.500000
max,9851.000000,10000.000000


The only true numerical column is number of enrollments. Such variable has a great distribution with a range of 

### 4.1. Statistical Analysis

In [ ]:
missing_counts = df.isnull().sum()

In [40]:
for table in tables_ddbb:
    print (f"# # # # # # {table} ({len(dict_df_clinical_trials[table])} records in total) # # # # # # ")
    print (f"Missing values:")
    df=pd.DataFrame(dict_df_clinical_trials[table].isna().sum(), columns=['Missing Values'])
    df['% missing'] = ((df['Missing Values'] / len(dict_df_clinical_trials[table])) * 100).round(2)

    df['type'] = dict_df_clinical_trials[table].dtypes
    display(df)
       

# # # # # # studies (10000 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
study_id,0,0.00,int64
nct_id,0,0.00,str
title,0,0.00,str
acronym,7188,71.88,str
status,0,0.00,str
phase,2455,24.55,str
study_type,0,0.00,str
start_date,103,1.03,object
completion_date,285,2.85,object
primary_completion_date,382,3.82,object


# # # # # # conditions (17750 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
condition_id,0,0.0,int64
study_id,0,0.0,int64
condition_name,0,0.0,str
mesh_term,17750,100.0,object


# # # # # # interventions (16715 records in total) # # # # # # 
Missing values:


...

,Missing Values,% missing,type
sponsor_id,0,0.00,int64
study_id,0,0.00,int64
agency,0,0.00,str
agency_class,23,0.15,str
lead_or_collaborator,0,0.00,str


# # # # # # locations (56916 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
location_id,0,0.00,int64
study_id,0,0.00,int64
facility,0,0.00,str
city,0,0.00,str
state,0,0.00,str
country,0,0.00,str
continent,20,0.04,str


# # # # # # study_design (9851 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
design_id,0,0.00,int64
study_id,0,0.00,int64
allocation,2397,24.33,str
intervention_model,2405,24.41,str
masking,2396,24.32,str
primary_purpose,2431,24.68,str
observational_model,7540,76.54,str


In [20]:
# --- IMPORTANT: Replace these with your actual MySQL database credentials ---
# Ensure your MySQL database is accessible from this Colab environment.

import os

# Uncomment the following two lines if you have a .env file with your credentials
# from dotenv import load_dotenv
# load_dotenv()

# Alternatively, set them directly as environment variables:
# Example placeholders. You MUST change these to your actual database details.
os.environ['DB_HOST'] = '127.0.0.1' # e.g., 'your_remote_db_host.com' or '127.0.0.1' if running locally
os.environ['DB_PORT'] = '3306' # Your specific MySQL port, often 3306
os.environ['DB_USER'] = 'root' # Your MySQL username
os.environ['DB_PASSWORD'] = 'rootpass' # Your MySQL password
os.environ['DB_NAME'] = 'clinicaltrials' # Your database name

print("Placeholder MySQL database credentials have been set. Please update them with your actual details.")
print("After updating, re-run the connection cell below (J1qAT_SARyIP).")

Placeholder MySQL database credentials have been set. Please update them with your actual details.
After updating, re-run the connection cell below (J1qAT_SARyIP).
